# Map / Reduce en python avec mrjob

## Prérequis : installation de mrjob avec Anaconda

### Ajout de conda-forge comme canal de distribution

Dans le navigateur Anaconda, 

1. cliquer sur `Channels` puis `Add`.
2. Dans la boite de dialogue, saisir `conda-forge` puis valider.
3. Rafraichir l'index en cliquant sur le bouton `Update index...`. 

Alternativement, en ligne de commande, selon la [documentation](https://conda-forge.org/) :

```
conda config --add channels conda-forge
conda config --set channel_priority strict
conda install <package-name> 
```

### Installation de mr-job

Dans la boite de recherche à côté du bouton `Update index...` saisir `mrjob` puis valider.

Alternativement, en ligne de commande, saisir `conda install -c conda-forge mrjob`

## Exemple canonique : le word frequency count

La [documentation](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#writing-your-first-job) fournit un premier exemple qui calcule et retourne le nombre de caractères, de mots et de lignes.

Afin de pouvoir exécuter l'exemple depuis le notebook nous 
- ajoutons l'instruction `%%file wordcount.py`
- déplaçons le `run`dans une autre cellule

(Explication : https://stackoverflow.com/questions/24701101/run-mrjob-from-ipython-notebook/33357944)

In [8]:
%%file wordfreqcount.py
from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):

    def mapper(self, _, line):
        yield "chars", len(line)
        yield "mots", len(line.split())
        yield "lignes", 1

    def reducer(self, key, values):
        yield key, sum(values)


Overwriting wordfreqcount.py


Ce code fournit 2 méthodes
- `mapper` génère, pour chaque ligne du texte, 3 couples (clé/valeur) dont les valeurs respectives sont le nombre de caractères, de mots et le nombre 1 (car on traite une ligne)
- `reducer` effectue, pour chaque clé, la somme des valeurs générées par le mapper pour chaque ligne - donc la somme des caractères, des mots et des lignes

Il nous reste à appeler le code depuis la cellule suivante. Nous nous appuyons sur la documentation qui explique comment lancer le programme [de manière programmatique](https://mrjob.readthedocs.io/en/latest/guides/runners.html#running-your-job-programmatically).

Nous ajoutons l'import du fichier généré dans la cellule précédente


In [9]:
from importlib import reload
import wordfreqcount
reload(wordfreqcount)

mr_job = wordfreqcount.MRWordFrequencyCount(args=['Data/Le Corbeau et le Renard.txt'])
with mr_job.make_runner() as runner:
     runner.run()
     for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


mots 131
lignes 18
chars 672


## Application au comptage des mots

Nous modifions l'exemple précédent de la manière suivante

- Le fichier généré s'appelle désormais comptage.py
- La classe s'appelle ComptageDesMots
- Un couple clé/valeur est généré pour chaque *mot* 

In [10]:
%%file comptage.py
import string
from mrjob.job import MRJob

class ComptageDesMots(MRJob):

    def mapper(self, _, ligne):
        ligneNettoyee = ligne.translate(str.maketrans('', '', string.punctuation))
        listeDeMots=ligneNettoyee.split()
        for mot in listeDeMots:
            yield mot, 1
    
    def reducer(self, key, values):
        yield key, sum(values)

Overwriting comptage.py


Comme pour l'exemple précédent, l'exécution est lancée depuis une autre cellule afin de respecter le fonctionnement d'Hadoop, où le code est *transmis* là où se situent les données.

In [11]:
from importlib import reload
import comptage
reload(comptage)

mr_job = comptage.ComptageDesMots(args=['Data/Le Corbeau et le Renard.txt'])
with mr_job.make_runner() as runner:
     runner.run()
     for key, value in mr_job.parse_output(runner.cat_output()):
        print(key, value)

No configs specified for inline runner


rapporte 1
sa 2
saisit 1
sans 1
se 1
fromage 2
hôtes 1
honteux 1
tard 1
tint 1
tomber 1
tout 1
un 5
vaut 1
voix 1
votre 2
vous 2
semblez 1
sen 1
sent 1
si 1
son 1
sur 1
joie 1
joli 1
lécoute 1
laisse 1
langage 1
proie 1
que 2
qui 1
quon 1
ramage 1
êtes 2
alléché 1
arbre 1
Lui 1
Maître 2
Mon 1
Monsieur 2
Phénix 1
Que 1
Renard 2
Sans 1
Se 1
Tenait 1
Vit 1
Vous 1
À 1
à 2
mais 1
me 1
mentir 1
montrer 1
mots 1
ne 2
ouvre 1
par 1
pas 1
perché 1
peu 2
plumage 1
plus 1
pour 1
près 1
prendrait 1
large 1
le 2
leçon 1
lodeur 1
ly 1
confus 1
dépens 1
de 3
des 1
Et 2
Faucon 1
Il 1
Jura 1
Le 2
aux 1
beau 1
bec 2
belle 1
bien 1
bois 1
bon 1
bonjour 1
ce 1
celui 1
ces 2
Apprenez 1
Cette 1
Corbeau 3
dit 1
doute 1
du 1
en 1
et 2
flatteur 1


# Conclusion

Le programme de ce notebook peut être exécuté sur un véritable cluster Hadoop et traiter plusieurs milliards de lignes comme il a pu traiter de la fable

Pouvez-vous finaliser le travail
1. en produisant la même sortie que l'exemple "python base", en n'affichant que le top 10 des mots les plus fréquents ?
2. ignorer la casse pour que chaque mot ne soit compté qu'une fois, qu'il apparaisse en minuscules ou majuscules

Appuyez-vous sur la [documentation](https://kite.com/python/docs/mrjob.job.MRJob.parse_output) de `parse-output` : 